In [47]:
import tweepy
import json
import pandas as pd
from textblob import TextBlob 
import re
import logging
import time

logger = logging.getLogger()

In [48]:
# Twitter API login function
def create_api():

    # login details
    api_key = 'TsTpQDhVRkQPI9pY8VsVQgF3o'
    api_secret = 'pR5adD28yuIa7f6f7mkBwsDMeutw2hwVoKl9aU9HtfochIDPB9'
    access_token = '1353358231939104770-r22ZfYRh6tHX0EZvk77TUDkr2eKs5N'
    access_token_secret = 'kPzb1hXVUfbpA5kaCbRUa0B2MersgGmZKfYvX3p2e852S'

    # login
    auth = tweepy.OAuthHandler(api_key, api_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth,
                     #wait_on_rate_limit=True
                    #waity_on_rate_limit_notify=True
                    )

    # confirm login ok
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error("Error creating API", exc_info=True)
        raise e
    logger.info("API created")
    return api

In [50]:
# function to add tweets to df

def add_tweets(query):
    # get and cycle over tweets
    for tweet in tweepy.Cursor(api.search,
                               q=query, # get tweets with certain keyword 
                               lang = 'en', # in english only
                               geocode="52.9548,1.1581,300km", # 300km radius centered on Nottingham
                              ).items(5):

        #add each tweet to the df
        data = [tweet.user.name, tweet.created_at, tweet.text]
        df.loc[len(df)] = data
    
    return (df)

In [51]:
# function to clean tweets
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)",
                            " ", tweet).split())
    
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(tweet) 
        
        # set sentiment 
        #if analysis.sentiment.polarity > 0: 
        #    return 'positive'
        #elif analysis.sentiment.polarity == 0: 
        #    return 'neutral'
        #else: 
        #    return 'negative'
        
        #note: 0 is neutral, >0 is positive, <0 is negative
        return analysis.sentiment.polarity

In [ ]:
def main():
    api = create_api()
    query = 'Lockdown'
    delay_secs = 60*15  #wait 15 minutes between queries
    
    #blank df to populate
    df = pd.DataFrame(columns = ['User','Date','Tweet'])
    
    while True:
        add_tweets(query)
        logger.info("Waiting...")
        time.sleep(delay_secs)

if __name__ == "__main__":
    main()

In [ ]:
# add column with cleaned tweets
df['TweetClean'] = df['Tweet'].apply(clean_tweet)

# add column with sentiment
df['Sentiment'] = df['TweetClean'].apply(get_tweet_sentiment)

In [ ]:
print(df.head())
df.plot(y='Sentiment', x='Date', kind = 'scatter')

In [58]:
import dash
import dash_core_components as dcc
import dash_html_components as html
########### Initiate the app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
application = app.server
app.title='Dash on AWS EB!'
########### Set up the layout
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),
    html.Div(children='''
            This is Dash running on Elastic Beanstalk.
        '''),
    dcc.Graph(
            id='example-graph',
            figure={
                'data': [
                    {'x': ['left', 'center', 'right'], 'y': [3,7,6], 'type': 'bar', 'name': 'category 1'},
                    {'x': ['left', 'center', 'right'], 'y': [4,2,5], 'type': 'bar', 'name': 'category 2'},
                ],
                'layout': {
                    'plot_bgcolor': 'lightgray',
                    'title': 'Graph Title',
                    'xaxis':{'title':'x-axis label'},
                    'yaxis':{'title':'y-axis label'},
                },
            }
        )
])
########### Run the app
app.run(debug=True)

AttributeError: 'Dash' object has no attribute 'run'

In [60]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.gapminder()
all_continents = df.continent.unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id="checklist",
        options=[{"label": x, "value": x} 
                 for x in all_continents],
        value=all_continents[3:],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="line-chart"),
])

@app.callback(
    Output("line-chart", "figure"), 
    [Input("checklist", "value")])
def update_line_chart(continents):
    mask = df.continent.isin(continents)
    fig = px.line(df[mask], 
        x="year", y="lifeExp", color='country')
    return fig

if __name__ == '__main__':
    application.run(debug=True, port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Restarting with windowsapi reloader


SystemExit: 1